#Collecting weather data from an API
**About the data**

In this notebook, we will be collecting daily weather data from the National Centers for Environmental Information (NCEI) API. We will use the Global Historical Climatology
Network - Daily (GHCND) data set; see the documentation here.

Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the
NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for the NCEI weather API to find the updated one.




**Using the NCEI API**

Paste your token below.

In [17]:
import requests

def make_req(endpoint, payload=None):
  return requests.get(
      f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
      headers={
          'token' : 'COGqDwKIRDvdlxMHVhxgqTsoJqtCoAfz'
      },params=payload
  )

In [18]:
response = make_req('datasets', {'startdate':'2018-10-01'})
response.status_code

# check if API is working

200

#Collect All Data Points for 2018 In NYC (Various Stations)
We can make a loop to query for all the data points one day at a time. Here we create a list of all the results:


In [31]:
import datetime

from IPython import display as dis

current = datetime.date(2018,1,1)
end = datetime.date(2019,1,1)

results = []

while current < end :
  dis.clear_output(wait=True)
  dis.display(f'gathering data for {str(current)}')

  response = make_req(
      'data',{
          'datasetid':'GHCND',
          'locationid':'CITY:US360019',
          'startdate':current,
          'enddate':current,
          'units':'metric',
          'limit':100 # limit set to 100 laptop can't handle :( (crashes)
      }
  )
  if response.ok:
    results.extend(response.json()['results'])

  current += datetime.timedelta(days=1)

'gathering data for 2018-12-31'

Now, we can create a dataframe with all this data. Notice there are multiple stations with values for each datatype on a given day. We don't know what the stations are, but we
can look them up and add them to the data:


In [33]:
import pandas as p

df=p.DataFrame(results)
df

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,0800",0.0
1,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,1050",0.0
2,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,1050",0.0
3,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,0920",0.0
4,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,0920",0.0
...,...,...,...,...,...
36095,2018-12-31T00:00:00,SNOW,GHCND:US1NJUN0003,",,N,0900",0.0
36096,2018-12-31T00:00:00,PRCP,GHCND:US1NJUN0010,",,N,0730",0.0
36097,2018-12-31T00:00:00,SNOW,GHCND:US1NJUN0010,",,N,0730",0.0
36098,2018-12-31T00:00:00,PRCP,GHCND:US1NJUN0014,",,N,0800",0.3


Save this data to a file:

In [38]:
df.to_csv('/content/sample_data/nycweather2k18.csv', index=False)

and write it to the database:

In [39]:
import sqlite3 as sq3

with sq3.connect('/content/weather.db') as connection:
  df.to_sql(
      'weather',connection, index=False, if_exists='replace'
  )

For learning about merging dataframes, we will also get the data mapping station IDs to information about the station:

In [42]:
response = make_req(
    'stations',{
        'datasetid':'GHCND',
        'locationid':'CITY:US360019',
        'limit':10
    }
)
# serached using the database(SQLite3) connection from previous cell ↑↑↑
stations = p.DataFrame(response.json()['results'])[['id','name','latitude','longitude','elevation']]
stations.to_csv('/content/sample_data/weather_stations.csv', index=False) # create a csv of the weather stations

with sq3.connect('/content/weather.db') as connection:
  stations.to_sql('stations',connection,index=False, if_exists='replace')

In [41]:
ws = p.read_csv('/content/sample_data/weather_stations.csv')
ws # weather stations

,id,name,latitude,longitude,elevation
0,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6
1,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.037788,-73.568176,6.4
2,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.921298,-74.001983,20.1
3,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.902694,-74.083358,16.8
4,GHCND:US1NJBG0003,"TENAFLY 1.3 W, NJ US",40.914670,-73.977500,21.6
5,GHCND:US1NJBG0005,"WESTWOOD 0.8 ESE, NJ US",40.983041,-74.015858,15.8
6,GHCND:US1NJBG0006,"RAMSEY 0.6 E, NJ US",41.058611,-74.134068,112.2
7,GHCND:US1NJBG0008,"SADDLE BROOK TWP 0.3 NNE, NJ US",40.907116,-74.093406,12.8
8,GHCND:US1NJBG0010,"RIVER VALE TWP 1.5 S, NJ US",40.991450,-74.012348,9.4
9,GHCND:US1NJBG0011,"NORTH ARLINGTON 0.7 NE, NJ US",40.794355,-74.119043,8.2
